# 학습목표
 1. flask를 이용한 REST API(GET, POST)실습
 2. 데이터분석 결과 REST API로 생성


* flask module
 - http://flask.pocoo.org/
 - micro-framework for python
 - jinja2 html template을 이용하여 rendering

In [19]:
from flask import Flask

# flask 객체 생성
app = Flask(__name__)

* plain text serving
 - 단순 텍스트를 반환

In [ ]:
# /txt_test url과 함수를 연결하는 decorator
@app.route('/txt_test')
def hello_plain():
    return 'plain text'

* json serving
 - json 형식으로 반환
 - API에서 가장 많이 사용되는 형태

In [20]:
from flask import jsonify

@app.route('/json_test')
def hello_json():
    data = {'name' : 'Aaron', 'family' : 'Byun'}
    return jsonify(data)

* html serving
 - 일반적인 웹페이지에 접속하는 과정이라고 생각하면 됨
 - 서버는 요청을 받아, 적절한 처리를 한뒤 html을 반환함

In [21]:
@app.route('/html_test')
def hello_html():
    # html file은 templates 폴더에 위치해야 함
    return render_template('simple.html')

* html template
 - jinja2 engine을 사용
 - {{}} 일반 값
 - {% for %} {% endfor %} 반복문
 - {% if %} {% elif %} {% else %} {% endif %} 조건문
 - simple2.html, app.py 참조 

* post method
 - crawling시, POST 요청으로 받는 url은 아래와 같이 구현됨
 - request.form dict를 이용하여 parameter 추출 가능

In [22]:
from flask import request

# POST임을 명시
@app.route('/register', methods=['POST'])
def register():
    
    # post parameter 가져오기
    username = request.form['username']
    password = request.form['password']
    
    return username

* konlpy 모듈
 - 한글 NLP(자연어 처리) 모듈
 - http://konlpy.org/en/v0.4.4/
 
* 설치 안내 
 - http://konlpy.org/en/v0.4.4/install/
 - mac에서 실행 시, Unable to load DLL 에러 발생 시 아래 링크 참고
 - https://github.com/konlpy/konlpy/issues/76
 
* 품사 (POS, part-of-speech) 비교표
 - https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0
 
* 형태소 (morph)
 - 언어에서 의미적으로 더이상 쪼갤 수 없는 가장 작은 단위

In [13]:
from konlpy.tag import Kkma, Hannanum, Komoran, Twitter
from konlpy.utils import pprint

kkma = Kkma()
hann = Hannanum()
komoran = Komoran()
twitter = Twitter()

ustr = u'안녕하세요 좋은 아침 입니다 식사는 하셨어요? 이따가 만납시다'

# 문장 단위 분리
pprint(kkma.sentences(ustr))

# 명사 추출
pprint(kkma.nouns(ustr))

# POS(Part Of Speech) 태깅
pprint(kkma.pos(ustr))

# 형태소 추출
pprint(kkma.morphs(ustr))

pprint(twitter.nouns(ustr))
pprint(hann.nouns(ustr))

print dir(twitter)


[안녕하세요,
 좋은 아침 입니다,
 식사는 하셨어요?,
 이따가 만납시다]
[안녕, 아침, 식사]
[(안녕, NNG),
 (하, XSV),
 (세요, EFN),
 (좋, VA),
 (은, ETD),
 (아침, NNG),
 (이, VCP),
 (ㅂ니다, EFN),
 (식사, NNG),
 (는, JX),
 (하, VV),
 (시, EPH),
 (었, EPT),
 (어요, EFN),
 (?, SF),
 (이따가, MAG),
 (만나, VV),
 (ㅂ시다, EFA)]
[안녕,
 하,
 세요,
 좋,
 은,
 아침,
 이,
 ㅂ니다,
 식사,
 는,
 하,
 시,
 었,
 어요,
 ?,
 이따가,
 만나,
 ㅂ시다]
[아침, 식사]
[안녕, 아침, 식사]
['__doc__', '__init__', '__module__', 'jki', 'morphs', 'nouns', 'phrases', 'pos', 'tagset']


* 연습문제) 
 1. 회원 가입 기능 구현
 2. 로그인 기능 구현
 3. 뉴스 검색 페이지 / API 구현(키워드 검색)
   - 해당 키워드를 포함하는 뉴스를 반환하는 페이지와 API 구현
   - 성능 향상을 위한 참고자료 https://docs.mongodb.com/v3.4/core/index-text/
 4. 실시간 / 배치 유사 뉴스 찾기 기능 구현
   - 실시간(real-time) : 요청이 오는 즉시 프로세싱하여 결과를 반환(느림, 최신정보 반영 가능)
   - 배치(batch)      : 주기적으로 결과를 생성하여 요청이 오면 해당 시점의 결과를 반환(빠름, 다소 정보 누락이 있을 수 있음)
   - 유사 뉴스 찾기
      - 단어(여기서는 간단히 하기 위해 명사) 벡터를 만들어 각각 다른 뉴스와 similarity 계산 후(cosine), 가장 높은 것을 반환
 5. 뉴스 컨텐츠 요약 기능 구현
   - 각 문서에서 핵심 키워드 몇개를 찾고, 핵심 키워드를 포함하는 횟수에 따라 문장 스코어링 후, 낮은 스코어의 문장을 버림
 6. 뉴스 카테고리 분류기 구현
   - 전체 문서 집합 단어에 대해, feature vector를 만들어 모델링